In [1]:
%autosave 10
import time

Autosaving every 10 seconds


In [2]:
import openpyxl
import pandas as pd

# Please ensure that the Length of the Final Cuts are rounded off considerably

In case of mismatch due to algorithm rounding off after the Pattern Generation sheet is obtained, it may be manually matched

In [3]:
DashBoard_Obj=pd.read_excel("DashBoard.xlsx","Objective",index_col=0)
print(DashBoard_Obj)

                                                    Value  \
Parameter                                                   
Problem                                                20   
Objective                                               4   
Number of Combinations to be Generated                 50   
Timelimit for each of the smallest problems            30   
Timelimit for the Large CSP problems                  500   
Maximum Number of Unique Standard Length usages         2   
Maximum Number of Unique Pattern usages per Sta...     50   

                                                                                              Remarks  
Parameter                                                                                              
Problem                                                                  Diameter of the Bars (in mm)  
Objective                                           Case Type 4 = Generating the Top Minimum Waste...  
Number of Combinations to be Gener

In [4]:
# Create a dictionary from the index and a specific column
Decisions = DashBoard_Obj['Value'].to_dict()
print(Decisions)
# Decisions["Maximum Number of (Unique) Standard Length usages"]

{'Problem': 20, 'Objective': 4, 'Number of Combinations to be Generated': 50, 'Timelimit for each of the smallest problems': 30, 'Timelimit for the Large CSP problems': 500, 'Maximum Number of Unique Standard Length usages': 2, 'Maximum Number of Unique Pattern usages per Standard Length': 50}


In [5]:
DashBoard_Dataset_Cuts=pd.read_excel("DashBoard.xlsx","Dataset_Cuts",index_col=0)
print(DashBoard_Dataset_Cuts)

                         Total Requirements
Lengths of Cuts Allowed                    
1.810                                     8
2.830                                  2134
2.950                                   954
3.030                                   332
3.892                                    48
3.911                                    36
3.951                                    48
3.960                                    48
4.230                                   332
4.332                                     8
4.344                                    24
4.364                                    29
4.503                                    24
4.730                                  1188
6.455                                    32
6.492                                    16
6.569                                    32
6.586                                    32
7.090                                     4
7.328                                    16
7.350                           

In [6]:
DashBoard_Dataset_StandardLengths=pd.read_excel("DashBoard.xlsx","Dataset_StandardLengths",index_col=0)
#print(type(DashBoard_Dataset_StandardLengths))
print(DashBoard_Dataset_StandardLengths)

                                Associated Cost
Total Standard Lengths Allowed                 
7                                      0.219911
8                                      0.251327
9                                      0.282743
10                                     0.314159
11                                     0.345575
12                                     0.376991


In [7]:
length_of_cuts_allowed = DashBoard_Dataset_Cuts.index.tolist()
print(length_of_cuts_allowed)

[1.81, 2.83, 2.95, 3.03, 3.892, 3.911, 3.951, 3.96, 4.23, 4.332, 4.344, 4.364, 4.503, 4.73, 6.455, 6.492, 6.569, 6.586, 7.09, 7.328, 7.35, 7.39, 7.626, 7.766]


In [8]:
requirements = DashBoard_Dataset_Cuts["Total Requirements"]
print(requirements)
#requirements[1.2]

Lengths of Cuts Allowed
1.810       8
2.830    2134
2.950     954
3.030     332
3.892      48
3.911      36
3.951      48
3.960      48
4.230     332
4.332       8
4.344      24
4.364      29
4.503      24
4.730    1188
6.455      32
6.492      16
6.569      32
6.586      32
7.090       4
7.328      16
7.350      16
7.390      16
7.626       9
7.766      24
Name: Total Requirements, dtype: int64


In [9]:
total_max_standardlengths_allowed = DashBoard_Dataset_StandardLengths.index.tolist()
print(total_max_standardlengths_allowed)

[7, 8, 9, 10, 11, 12]


In [10]:
combinations = range(1,int(Decisions["Number of Combinations to be Generated"])+1)
print(combinations)

range(1, 51)


In [11]:
#max_time_in_seconds = Decisions["Timelimit for each of the smallest problems"]

#Timelimit_for_each_final_CSP_problem_minimizing_WCSP = Decisions["Timelimit for the Large CSP problems"]

# Generating Combinations Depending on Objectives

In [12]:
import gurobipy as gp
from gurobipy import GRB

In [13]:
start_time_is_this=time.time()
print(start_time_is_this)

1702799068.4521801


# Constraints dependant on Objectives

In [14]:
if Decisions["Objective"] == 4:
    M = 10000000
    X_Standards = {}
    
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        
        for c in combinations:
            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                if cc < c:
                    for l in length_of_cuts_allowed:
                        m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                        g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                        prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                        prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                        prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                        prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                        prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                        prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                    prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            waste_generated = prob.objVal

            for l in length_of_cuts_allowed:
                X_store[c, l] = x[l].x
                if X_store[c, l] > 0:
                    print("\t X_store[", c, ",", l, "] = ", X_store[c, l])

            print("\t \t Total Waste Generated: ", waste_generated)
            X_store[c, "Waste Objective"] = waste_generated
            print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem of Standard length: ", max_length)

        X_Standards[max_length] = X_store.copy()

Max Standard Length Allowed:  7
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-10
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 24 columns and 24 nonzeros
Model fingerprint: 0xe923c73d
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 8e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 7e+00]
Found heuristic solution: objective 1.5700000
Presolve removed 0 rows and 14 columns
Presolve time: 0.00s
Presolved: 1 rows, 10 columns, 10 nonzeros
Variable types: 0 continuous, 10 integer (6 binary)
Found heuristic solution: objective 0.0100000

Root relaxation: objective 0.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Curr

Best objective 7.800000000000e-02, best bound 7.800000000000e-02, gap 0.0000%
	 X_store[ 4 , 3.03 ] =  1.0
	 X_store[ 4 , 3.892 ] =  1.0
	 	 Total Waste Generated:  0.07800000000000029
	 	 Total Time Taken:  0.1  seconds, for this smallest problem of Standard length:  7
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 581 rows, 216 columns and 1464 nonzeros
Model fingerprint: 0xfdb350b2
Variable types: 96 continuous, 120 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 552 rows and 198 columns
Presolve time: 0.01s
Presolved: 29 rows, 18 columns, 98 nonzeros
Variable types: 0 continuous, 18 integer (14 binary)
Found heuristic solution: objective 0.350

Found heuristic solution: objective 0.3500000

Root relaxation: objective 0.000000e+00, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    3    0.35000    0.00000   100%     -    0s
H    0     0                       0.1580000    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.15800    0.00000   100%     -    0s
     0     0    0.00000    0    8    0.15800    0.00000   100%     -    0s
     0     0    0.00000    0    8    0.15800    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.15800    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.15800    0.00000   100%     -    0s
     0     0    0.00000    0    8    0.15800    0.00000   100%     -    0s
     0     0    0.00000    0    8    0.15800    0.00000   100%     -    0s
     0     0    0.00000    0    8    0.

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    1    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    8    0.35000    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.35000    0.00000   100%     -    0s
     0     0    0.01000    0    1    0.35000    0.01000  97.1%     -    0s
     0     0    0.01000    0    4    0.35000    0.01000  97.1%     -    0s
     0     0    0.01450    0    7    0.35000    0.01450  95.9%     -    0s
     0     0    0.02633    0   15    0.35000    0.02633  92.5%     -    0s
     0     0    0.02960    0   13    0.35000    0.02960  91.5%     -    0s
     0     0    0.03450    0   12    0.35000    0.03450  90.1%     -    0s
     0     0    0.05000    0   10    0.35000    0.05000  85.7%     -    0s
     0     0    0.0545

     0     0    0.00000    0    1    0.43000    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.43000    0.00000   100%     -    0s
     0     0    0.00000    0   12    0.43000    0.00000   100%     -    0s
     0     0    0.00000    0    7    0.43000    0.00000   100%     -    0s
     0     0    0.00000    0    9    0.43000    0.00000   100%     -    0s
     0     0    0.00000    0    8    0.43000    0.00000   100%     -    0s
     0     0    0.00000    0    8    0.43000    0.00000   100%     -    0s
     0     0    0.01000    0    5    0.43000    0.01000  97.7%     -    0s
     0     0    0.01450    0    8    0.43000    0.01450  96.6%     -    0s
     0     0    0.02914    0   18    0.43000    0.02914  93.2%     -    0s
     0     0    0.02914    0   19    0.43000    0.02914  93.2%     -    0s
     0     0    0.03675    0   17    0.43000    0.03675  91.5%     -    0s
     0     0    0.03675    0   21    0.43000    0.03675  91.5%     -    0s
     0     0    0.03675  

...

     0     0    0.00000    0    1    0.12000    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.12000    0.00000   100%     -    0s
     0     2    0.00000    0    1    0.12000    0.00000   100%     -    0s
H  442    29                       0.1150000    0.00000   100%   7.7    0s

Cutting planes:
  Learned: 7
  Cover: 2
  Implied bound: 3
  Clique: 3
  Relax-and-lift: 3

Explored 1067 nodes (9684 simplex iterations) in 0.76 seconds (0.23 work units)
Thread count was 4 (of 4 available processors)

Solution count 3: 0.115 0.12 0.82 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.150000000000e-01, best bound 1.150000000000e-01, gap 0.0000%
	 X_store[ 35 , 1.81 ] =  3.0
	 X_store[ 35 , 6.455 ] =  1.0
	 	 Total Waste Generated:  0.11500000000000021
	 	 Total Time Taken:  0.8  seconds, for this smallest problem of Standard length:  12
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction s


Optimal solution found (tolerance 1.00e-04)
Best objective 1.200000000000e-01, best bound 1.200000000000e-01, gap 0.0000%
	 X_store[ 38 , 3.96 ] =  3.0
	 	 Total Waste Generated:  0.120000000000001
	 	 Total Time Taken:  1.1  seconds, for this smallest problem of Standard length:  12
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5511 rows, 1848 columns and 13704 nonzeros
Model fingerprint: 0xbfe4a034
Variable types: 912 continuous, 936 integer (912 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 4950 rows and 1650 columns
Presolve time: 0.08s
Presolved: 561 rows, 198 columns, 2330 nonzeros
Variable types: 0 continuous, 198 integer (128 binary)
Found heurist

     0     0    0.00000    0    6    0.14400    0.00000   100%     -    0s
     0     0    0.00000    0    4    0.14400    0.00000   100%     -    0s
     0     0    0.00000    0    4    0.14400    0.00000   100%     -    0s
     0     2    0.00000    0    4    0.14400    0.00000   100%     -    0s
*   85    31              27       0.1370000    0.00000   100%   7.2    0s
*  321    52              19       0.1290000    0.00000   100%   9.6    0s

Cutting planes:
  Gomory: 1
  Cover: 4

Explored 1025 nodes (11936 simplex iterations) in 0.82 seconds (0.25 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 0.129 0.137 0.144 0.416 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.290000000000e-01, best bound 1.290000000000e-01, gap 0.0000%
	 X_store[ 42 , 3.951 ] =  1.0
	 X_store[ 42 , 3.96 ] =  2.0
	 	 Total Waste Generated:  0.12899999999999956
	 	 Total Time Taken:  0.8  seconds, for this smallest problem of Standard length:  12
Gurobi Optimizer v

H  993    24                       0.1380000    0.00000   100%  10.6    0s

Cutting planes:
  Learned: 27
  Cover: 14
  Implied bound: 4
  MIR: 2
  RLT: 6

Explored 1225 nodes (13678 simplex iterations) in 1.12 seconds (0.31 work units)
Thread count was 4 (of 4 available processors)

Solution count 6: 0.138 0.144 0.156 ... 0.428

Optimal solution found (tolerance 1.00e-04)
Best objective 1.380000000000e-01, best bound 1.380000000000e-01, gap 0.0000%
	 X_store[ 45 , 3.951 ] =  2.0
	 X_store[ 45 , 3.96 ] =  1.0
	 	 Total Waste Generated:  0.1379999999999999
	 	 Total Time Taken:  1.1  seconds, for this smallest problem of Standard length:  12
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6526 rows, 2184 columns and 16224 nonzeros
Model fingerprint: 0xf63e4e5d
Variable types: 1080 continuous, 11

Found heuristic solution: objective 0.2570000

Root relaxation: objective 0.000000e+00, 257 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    8    0.25700    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.25700    0.00000   100%     -    0s
     0     0    0.00000    0    6    0.25700    0.00000   100%     -    0s
     0     2    0.00000    0    6    0.25700    0.00000   100%     -    0s
*   51    45               9       0.1690000    0.00000   100%   7.8    0s
H  251    60                       0.1660000    0.00000   100%  10.3    0s
*  430    61              17       0.1500000    0.00000   100%  11.3    0s
*  703    50              22       0.1470000    0.00000   100%  11.9    0s

Cutting planes:
  Learned: 13
  Gomory: 1
  Cover: 6

Explored 990 nodes (12548 simplex iterations) in 1.02 seco

In [15]:
max_combination_for_each_standard = {}

if Decisions["Objective"] == 4:
    for max_length in total_max_standardlengths_allowed:
        print("Max Standard Length Allowed: ", max_length)
        X_store = {}
        combination_previous_end_number = Decisions["Number of Combinations to be Generated"]
        current_combination_number = combination_previous_end_number

        for ll in length_of_cuts_allowed:
            if ll > max_length:
                continue

            current_combination_number += 1

            # Create a new Gurobi model
            prob = gp.Model("Stock Cutting with multiple Master Lengths for Minimum Waste")

            # Variables
            x = {}
            m = {}
            g = {}

            for l in length_of_cuts_allowed:
                x[l] = prob.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{l}]')

            # Objective: Minimize Waste
            prob.setObjective(max_length - gp.quicksum(x[l] * l for l in length_of_cuts_allowed), sense=GRB.MINIMIZE)

            # Constraint 0
            prob.addConstr(x[ll] >= 1)

            # Constraint 1
            prob.addConstr(gp.quicksum(x[l] * l for l in length_of_cuts_allowed) <= max_length)

            # Constraint 2
            for cc in combinations:
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_Standards[max_length][cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_Standards[max_length][cc, l])

                    prob.addConstr(x[l] - X_Standards[max_length][cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_Standards[max_length][cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_Standards[max_length][cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_Standards[max_length][cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            for cc in range(combination_previous_end_number + 1, current_combination_number):
                for l in length_of_cuts_allowed:
                    m[cc, l] = prob.addVar(lb=0, name=f'm[{cc},{l}]')
                    g[cc, l] = prob.addVar(vtype=GRB.BINARY, name=f'g[{cc},{l}]')

                    prob.addConstr(m[cc, l] >= X_store[cc, l] - x[l])
                    prob.addConstr(m[cc, l] >= x[l] - X_store[cc, l])

                    prob.addConstr(x[l] - X_store[cc, l] <= M * (1 - g[cc, l]))
                    prob.addConstr(X_store[cc, l] - x[l] <= M * g[cc, l])

                    prob.addConstr(m[cc, l] <= X_store[cc, l] - x[l] + M * (1 - g[cc, l]))
                    prob.addConstr(m[cc, l] <= x[l] - X_store[cc, l] + M * g[cc, l])

                prob.addConstr(gp.quicksum(m[cc, l] for l in length_of_cuts_allowed) >= 1)

            # Solve the problem
            #prob.Params.TimeLimit = max_time_in_seconds
            prob.optimize()
            
            
            
            if prob.SolCount:
                waste_generated = prob.objVal

                for l in length_of_cuts_allowed:
                    X_store[current_combination_number, l] = x[l].x
                    if X_store[current_combination_number, l] > 0:
                        print("\t X_store[", current_combination_number, ",", l, "] = ", X_store[current_combination_number, l])

                print("\t \t Total Waste Generated: ", waste_generated,
                      " during mandatory unique usage of cut-length: ", ll, "in standard length: ", max_length)
                X_store[current_combination_number, "Waste Objective"] = waste_generated
                print("\t \t Total Time Taken: ", round(prob.Runtime, 1), " seconds, for this smallest problem")
            else:
                current_combination_number-=1

        max_combination_for_each_standard[max_length] = current_combination_number
        X_Standards[max_length].update(X_store)

Max Standard Length Allowed:  7
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7252 rows, 2424 columns and 18025 nonzeros
Model fingerprint: 0x56185e82
Variable types: 1200 continuous, 1224 integer (1200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 7252 rows and 2424 columns
Presolve time: 0.05s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.08 seconds (0.03 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 5.19 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.190000000000e+00, best bound 5.190000000000e+00, gap 0.0000%
	 X_store[ 51 , 1.81 ] =  1.0
	 	 Total Waste Generated:  

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 7824 rows and 2616 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7832 rows, 2616 columns and 19465 nonzeros
Model fingerprint: 0xb2612f60
Variable types: 1296 continuous, 1320 integer (1296 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 7824 rows and 2616 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.05 secon

  RHS range        [1e+00, 1e+07]
Presolve removed 7134 rows and 2377 columns
Presolve time: 0.04s
Presolved: 118 rows, 47 columns, 428 nonzeros
Variable types: 0 continuous, 47 integer (35 binary)
Found heuristic solution: objective 1.3500000

Root relaxation: objective 0.000000e+00, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   13    1.35000    0.00000   100%     -    0s
     0     0    0.00000    0    9    1.35000    0.00000   100%     -    0s
     0     0    0.00000    0    6    1.35000    0.00000   100%     -    0s
     0     0    0.00000    0    7    1.35000    0.00000   100%     -    0s
     0     0    0.00000    0    8    1.35000    0.00000   100%     -    0s
     0     0    0.00000    0    7    1.35000    0.00000   100%     -    0s
     0     0    0.00000    0   16    1.35000    0.00000   100%    

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7832 rows, 2616 columns and 19465 nonzeros
Model fingerprint: 0xc4233a33
Variable types: 1296 continuous, 1320 integer (1296 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 7832 rows and 2616 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 1.078 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.078000000000e+00, best bound 1.078000000000e+00, gap 0.0000%
	 X_store[ 55 , 3.03 ] =  1.0
	 X_store[ 55 , 3.892 ] =  1.0
	 	 Total Waste Generated:  

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 8847 rows, 2952 columns and 21985 nonzeros
Model fingerprint: 0xab3dff24
Variable types: 1464 continuous, 1488 integer (1464 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 8847 rows and 2952 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 1.826 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.826000000000e+00, best bound 1.826000000000e+00, gap 0.0000%
	 X_store[ 62 , 1.81 ] =  1.0
	 X_store[ 62 , 4.364 ] =  1.0
	 	 Total Waste Generated:  1.8260000000000005  during mandatory unique usage of cut-length:  4.364 in standard length:  8
	 	 Total Time Taken:  0.1  seconds, for this sma

...


Optimize a model with 8267 rows, 2760 columns and 20545 nonzeros
Model fingerprint: 0x46322c46
Variable types: 1368 continuous, 1392 integer (1368 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 8130 rows and 2703 columns
Presolve time: 0.05s
Presolved: 137 rows, 57 columns, 500 nonzeros
Variable types: 0 continuous, 57 integer (42 binary)
Found heuristic solution: objective 1.1790000
Found heuristic solution: objective 0.8000000
Found heuristic solution: objective 0.4600000

Root relaxation: objective 0.000000e+00, 49 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    4    0.46000    0.00000   100%     -    0s
H    0     0                       0.2180000    0.00000   100%     

Found heuristic solution: objective 1.2060000
Found heuristic solution: objective 0.4160000
Found heuristic solution: objective 0.2620000

Root relaxation: objective 1.776357e-15, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    1    0.26200    0.00000   100%     -    0s
     0     0    0.00000    0    5    0.26200    0.00000   100%     -    0s
H    0     0                       0.2030000    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.20300    0.00000   100%     -    0s
     0     0    0.00000    0    2    0.20300    0.00000   100%     -    0s
     0     0    0.00000    0    1    0.20300    0.00000   100%     -    0s
     0     0    0.00000    0    9    0.20300    0.00000   100%     -    0s
     0     0    0.00000    0    3    0.20300    0.00000   100%     -    0s
     0     0    0.0000

     0     0    0.00000    0    8    0.28000    0.00000   100%     -    0s
     0     0    0.03000    0    8    0.28000    0.03000  89.3%     -    0s
     0     0    0.05000    0    6    0.28000    0.05000  82.1%     -    0s
H    0     0                       0.2100000    0.05000  76.2%     -    0s
     0     0    0.05000    0    7    0.21000    0.05000  76.2%     -    0s
*    0     0               0       0.1800000    0.18000  0.00%     -    0s

Cutting planes:
  Learned: 2
  Implied bound: 1
  Clique: 2
  MIR: 3
  Zero half: 1

Explored 1 nodes (114 simplex iterations) in 0.14 seconds (0.02 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 0.18 0.21 0.28 0.62 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.800000000000e-01, best bound 1.800000000000e-01, gap 0.0000%
	 X_store[ 64 , 4.73 ] =  1.0
	 X_store[ 64 , 7.09 ] =  1.0
	 	 Total Waste Generated:  0.17999999999999972  during mandatory unique usage of cut-length:  4.73 in standard length


Root relaxation: objective 6.241935e-02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.06242    0    1    0.27000    0.06242  76.9%     -    0s
     0     0 infeasible    0         0.27000    0.27000  0.00%     -    0s

Explored 1 nodes (5 simplex iterations) in 0.10 seconds (0.02 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 0.27 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.700000000000e-01, best bound 2.700000000000e-01, gap 0.0000%
	 X_store[ 69 , 1.81 ] =  1.0
	 X_store[ 69 , 2.83 ] =  1.0
	 X_store[ 69 , 7.09 ] =  1.0
	 	 Total Waste Generated:  0.2699999999999996  during mandatory unique usage of cut-length:  7.09 in standard length:  12
	 	 Total Time Taken:  0.1  seconds, for this smallest problem
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU mod

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.2740000    0.27400  0.00%     -    0s

Explored 1 nodes (0 simplex iterations) in 0.09 seconds (0.02 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 0.274 0.614 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.740000000000e-01, best bound 2.740000000000e-01, gap 0.0000%
	 X_store[ 74 , 3.96 ] =  1.0
	 X_store[ 74 , 7.766 ] =  1.0
	 	 Total Waste Generated:  0.2740000000000009  during mandatory unique usage of cut-length:  7.766 in standard length:  12
	 	 Total Time Taken:  0.1  seconds, for this smallest problem


In [16]:
import winsound
winsound.Beep(500,1000)
max_combination_for_each_standard

{7: 54, 8: 68, 9: 74, 10: 74, 11: 74, 12: 74}

In [17]:
wb=openpyxl.Workbook()

sheet_for_Combinations = wb.active

row_number=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Length of Cut"

column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value="Combination "+str(column_number-1)
        column_number+=1




for l in length_of_cuts_allowed:
    row_number+=1
    sheet_for_Combinations.cell(row=row_number,column=1).value= l
    column_number=2
    for i,max_length in enumerate(total_max_standardlengths_allowed):
        #for c in range(1,current_combination_number+1):
        for c in range(1,max_combination_for_each_standard[max_length]+1):
            sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,l]
            column_number+=1
        
        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Standard Max Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= max_length
        column_number+=1
        

        
row_number+=1
sheet_for_Combinations.cell(row=row_number,column=1).value= "Wasted Length"
column_number=2
for i,max_length in enumerate(total_max_standardlengths_allowed):
    #for c in range(1,current_combination_number+1):
    for c in range(1,max_combination_for_each_standard[max_length]+1):
        sheet_for_Combinations.cell(row=row_number,column= column_number).value= X_Standards[max_length][c,"Waste Objective"]
        column_number+=1

In [18]:
#file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+str(max_time_in_seconds)+".xlsx"
file_path = "./Problem_"+str(Decisions["Problem"])+"__CombinationGeneration_"+str(Decisions["Number of Combinations to be Generated"])+"__TimeLimit_"+".xlsx"
wb.save(file_path)
print(file_path)

./Problem_20__CombinationGeneration_50__TimeLimit_.xlsx


In [19]:
X_Standards

{7: {(1, 1.81): 0.0,
  (1, 2.83): 0.0,
  (1, 2.95): 0.0,
  (1, 3.03): 1.0,
  (1, 3.892): 0.0,
  (1, 3.911): 0.0,
  (1, 3.951): 0.0,
  (1, 3.96): 1.0,
  (1, 4.23): -0.0,
  (1, 4.332): -0.0,
  (1, 4.344): -0.0,
  (1, 4.364): -0.0,
  (1, 4.503): -0.0,
  (1, 4.73): 0.0,
  (1, 6.455): -0.0,
  (1, 6.492): -0.0,
  (1, 6.569): -0.0,
  (1, 6.586): 0.0,
  (1, 7.09): 0.0,
  (1, 7.328): 0.0,
  (1, 7.35): 0.0,
  (1, 7.39): 0.0,
  (1, 7.626): 0.0,
  (1, 7.766): 0.0,
  (1, 'Waste Objective'): 0.009999999999999787,
  (2, 1.81): 0.0,
  (2, 2.83): 0.0,
  (2, 2.95): 0.0,
  (2, 3.03): 1.0,
  (2, 3.892): 0.0,
  (2, 3.911): 0.0,
  (2, 3.951): 1.0,
  (2, 3.96): 0.0,
  (2, 4.23): 0.0,
  (2, 4.332): 0.0,
  (2, 4.344): -0.0,
  (2, 4.364): -0.0,
  (2, 4.503): 0.0,
  (2, 4.73): 0.0,
  (2, 6.455): 0.0,
  (2, 6.492): -0.0,
  (2, 6.569): 0.0,
  (2, 6.586): 0.0,
  (2, 7.09): 0.0,
  (2, 7.328): 0.0,
  (2, 7.35): 0.0,
  (2, 7.39): 0.0,
  (2, 7.626): 0.0,
  (2, 7.766): 0.0,
  (2, 'Waste Objective'): 0.019000000000000128

In [20]:
for max_length in total_max_standardlengths_allowed:
    #print(max_length)
    print(X_Standards[max_length])

{(1, 1.81): 0.0, (1, 2.83): 0.0, (1, 2.95): 0.0, (1, 3.03): 1.0, (1, 3.892): 0.0, (1, 3.911): 0.0, (1, 3.951): 0.0, (1, 3.96): 1.0, (1, 4.23): -0.0, (1, 4.332): -0.0, (1, 4.344): -0.0, (1, 4.364): -0.0, (1, 4.503): -0.0, (1, 4.73): 0.0, (1, 6.455): -0.0, (1, 6.492): -0.0, (1, 6.569): -0.0, (1, 6.586): 0.0, (1, 7.09): 0.0, (1, 7.328): 0.0, (1, 7.35): 0.0, (1, 7.39): 0.0, (1, 7.626): 0.0, (1, 7.766): 0.0, (1, 'Waste Objective'): 0.009999999999999787, (2, 1.81): 0.0, (2, 2.83): 0.0, (2, 2.95): 0.0, (2, 3.03): 1.0, (2, 3.892): 0.0, (2, 3.911): 0.0, (2, 3.951): 1.0, (2, 3.96): 0.0, (2, 4.23): 0.0, (2, 4.332): 0.0, (2, 4.344): -0.0, (2, 4.364): -0.0, (2, 4.503): 0.0, (2, 4.73): 0.0, (2, 6.455): 0.0, (2, 6.492): -0.0, (2, 6.569): 0.0, (2, 6.586): 0.0, (2, 7.09): 0.0, (2, 7.328): 0.0, (2, 7.35): 0.0, (2, 7.39): 0.0, (2, 7.626): 0.0, (2, 7.766): 0.0, (2, 'Waste Objective'): 0.019000000000000128, (3, 1.81): -0.0, (3, 2.83): -0.0, (3, 2.95): -0.0, (3, 3.03): 1.0, (3, 3.892): 0.0, (3, 3.911): 1.0,

# Main CSProblem starts here:~

(designed to also run separately from the above program, be cautious about the intermediate file name changes if done manually)

In [21]:
import gurobipy as gp
from gurobipy import GRB

from math import ceil

In [22]:
patterns_df = pd.read_excel(file_path, index_col=0)
#print(patterns_df)

In [23]:
# Get the last two rows into a new DataFrame
last_two_rows = patterns_df.tail(2).copy()  # tail(2) gets the last 2 rows

# Remove the last two rows from the original DataFrame
new_df = patterns_df.drop(patterns_df.tail(2).index)

In [24]:
# Find unique elements in the first row (index 0)
unique_elements_row = last_two_rows.iloc[0].unique()

print("Unique elements in the first row of last_two_rows:")
print(unique_elements_row)

Unique elements in the first row of last_two_rows:
[ 7.  8.  9. 10. 11. 12.]


In [25]:
all_length_of_cuts_allowed = new_df.index.tolist() # we could have used length_of_cuts_allowed, but here we take data from the Excel Sheet as we may want to change something for Later (in the Excel Sheet manually) which is however not recommended nor necessary
all_length_of_cuts_allowed

[1.81,
 2.83,
 2.95,
 3.03,
 3.892,
 3.911,
 3.951,
 3.96,
 4.23,
 4.332,
 4.344,
 4.364,
 4.503,
 4.73,
 6.455,
 6.492,
 6.569,
 6.586,
 7.09,
 7.328,
 7.35,
 7.39,
 7.626,
 7.766]

# Waste, Cost, Standard, Pattern: The WCSP sequence of Minimization (4 sequential problems

In [26]:
# Constraints
sl_key__value_c_mapping_dict={}
set_of_encountered_sl=set()
for c in patterns_df:
    sl = last_two_rows[c]["Standard Max Length"]
    if sl in set_of_encountered_sl:
        sl_key__value_c_mapping_dict[sl].add(c)
    else:
        sl_key__value_c_mapping_dict[sl]={c}
        set_of_encountered_sl.add(sl)
    
#print(sl_key__value_c_mapping_dict)

# WASTE MINIMIZATION

In [27]:
# Create a Gurobi model
CSProb = gp.Model("Stock Cutting with Waste Minimization")

# Variables
x = {}

for c in patterns_df:
    x[c] = CSProb.addVar(lb=0, vtype=GRB.INTEGER, name=f'x[{c}]')


# Objective: Minimize Waste
CSProb.setObjective(gp.quicksum(x[c] * last_two_rows[c]["Wasted Length"] for c in patterns_df), sense=GRB.MINIMIZE)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb.addConstr(gp.quicksum(x[c] * new_df[c][l] for c in patterns_df) >= requirements[l])





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb.Params.MIPFocus=2


# Optimize the model
CSProb.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 24 rows, 418 columns and 914 nonzeros
Model fingerprint: 0x2ed22838
Variable types: 0 continuous, 418 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e-03, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+03]
Found heuristic solution: objective 1016.2860000
Presolve

# COST MINIMIZATION

In [28]:
# Create a new Gurobi model
CSProb2 = gp.Model("Stock Cutting with Cost Minimization")

# Variables
x_2 = {}

for c in patterns_df:
    x_2[c] = CSProb2.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_2[{c}]')



# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb2.addConstr(gp.quicksum(x_2[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000, name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb2.addConstr(gp.quicksum(x_2[c] * new_df[c][l] for c in patterns_df) >= requirements[l])


# Objective 2
CSProb2.setObjective(gp.quicksum(x_2[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][last_two_rows[c]["Standard Max Length"]] for c in patterns_df),
                     sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb2.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb2.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb2.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb2.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb2.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb2.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb2.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb2.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb2.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb2.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb2.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb2.Params.MIPFocus=2


# Optimize the model
CSProb2.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 25 rows, 418 columns and 1330 nonzeros
Model fingerprint: 0x2ea310d1
Variable types: 0 continuous, 418 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-03, 6e+00]
  Objective range  [2e-01, 4e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+03]
Presolve removed 0 rows and 49 columns
Presolve time: 0.

# STANDARDs MINIMIZATION

In [29]:
# STANDARDs MINIMIZATION
M=100000

# Create a new Gurobi model
CSProb3 = gp.Model("Stock Cutting with Unique Standard length usage minimization")

#Debugging Mode
CSProb3.setParam('OutputFlag', 1)

# Variables
x_3 = {}
b_3 = {}

for c in patterns_df:
    x_3[c] = CSProb3.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_3[{c}]')

for sl in unique_elements_row:
    b_3[float(sl)] = CSProb3.addVar(vtype=GRB.BINARY, name=f'b_3[{sl}]')

    
    
    
# Constraints

for sl in unique_elements_row:
    CSProb3.addConstr( gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_3[float(sl)])
    CSProb3.addConstr( M*b_3[float(sl)] >= gp.quicksum(x_3[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    

    

# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000,name="Obj1 as Const.")


for l in all_length_of_cuts_allowed:
    CSProb3.addConstr(gp.quicksum(x_3[c] * new_df[c][l] for c in patterns_df) >= requirements[l],name=f"Requirement const. for {l}")


# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb3.addConstr(gp.quicksum(x_3[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000, name="Obj2 as Const.")


# New Objective 3 to Minimize the number of Unique Standard Length Usage
CSProb3.setObjective(gp.quicksum(b_3[float(sl)] for sl in unique_elements_row),sense=GRB.MINIMIZE)






# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb3.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb3.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb3.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb3.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb3.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb3.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb3.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb3.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb3.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb3.Params.MIPGap = 0
print("Assigned MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb3.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb3.Params.MIPFocus=2

#CSProb3.Params.NumericFocus = 3  # Experiment with different values (0, 1, 2, 3)


# Optimize the model
CSProb3.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Assigned MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 38 rows, 424 columns and 2596 nonzeros
Model fingerprint: 0x962b834d
Variable types: 0 continuous, 424 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+03]
Presolve added 6 rows and 0 columns
Presolve removed 0 r

In [30]:
# IMPORTANT

# #Identify Infeasible Constraints:
# #Use the infeasibility information (model.IIS) to identify the constraints causing infeasibility. This can be done as follows:

# CSProb3.computeIIS()
# print('IIS constraints:')
# for constr in CSProb3.getConstrs():
#     if constr.IISConstr:
#         print(constr.ConstrName)

In [31]:
# Print variable values
for var in CSProb3.getVars():
    print(f'{var.varName}: {var.x}')

# # Print constraint values
# for constr in CSProb3.getConstrs():
#     print(f'{constr.ConstrName}: {constr.Pi} (dual value)')

# Print objective value
print(f'Objective Value: {CSProb3.objVal}')

x_3[Combination 1]: -0.0
x_3[Combination 2]: -0.0
x_3[Combination 3]: -0.0
x_3[Combination 4]: -0.0
x_3[Combination 5]: -0.0
x_3[Combination 6]: -0.0
x_3[Combination 7]: -0.0
x_3[Combination 8]: -0.0
x_3[Combination 9]: -0.0
x_3[Combination 10]: -0.0
x_3[Combination 11]: -0.0
x_3[Combination 12]: -0.0
x_3[Combination 13]: -0.0
x_3[Combination 14]: -0.0
x_3[Combination 15]: -0.0
x_3[Combination 16]: -0.0
x_3[Combination 17]: -0.0
x_3[Combination 18]: -0.0
x_3[Combination 19]: -0.0
x_3[Combination 20]: -0.0
x_3[Combination 21]: -0.0
x_3[Combination 22]: -0.0
x_3[Combination 23]: -0.0
x_3[Combination 24]: -0.0
x_3[Combination 25]: -0.0
x_3[Combination 26]: -0.0
x_3[Combination 27]: -0.0
x_3[Combination 28]: -0.0
x_3[Combination 29]: -0.0
x_3[Combination 30]: -0.0
x_3[Combination 31]: -0.0
x_3[Combination 32]: -0.0
x_3[Combination 33]: -0.0
x_3[Combination 34]: -0.0
x_3[Combination 35]: -0.0
x_3[Combination 36]: -0.0
x_3[Combination 37]: -0.0
x_3[Combination 38]: -0.0
x_3[Combination 39]: 

# PATTERNs MINIMIZATION

In [32]:
# Create a new Gurobi model
CSProb4 = gp.Model("Stock Cutting with Minimum Unique Pattern usage")

# Variables
x_4 = {}
y_4 = {}
b_4 = {}

for c in patterns_df:
    x_4[c] = CSProb4.addVar(lb=0, vtype=GRB.INTEGER, name=f'x_4[{c}]')
    y_4[c] = CSProb4.addVar(vtype=GRB.BINARY, name=f'y_4[{c}]')

for sl in unique_elements_row:
    b_4[sl] = CSProb4.addVar(vtype=GRB.BINARY, name=f'b_4[{sl}]')

# Constraints
for c in patterns_df:
    #CSProb4.addConstr(x_4[c] >= b_4[last_two_rows[c]["Standard Max Length"]]) #Wrong
    CSProb4.addConstr(x_4[c] >= y_4[c])
    #CSProb4.addConstr(M * b_4[last_two_rows[c]["Standard Max Length"]] >= x_4[c]) #Wrong
    CSProb4.addConstr(M * y_4[c] >= x_4[c])

    
# Constraints
for sl in unique_elements_row:
    CSProb4.addConstr( gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]) >= b_4[float(sl)])
    CSProb4.addConstr( M*b_4[float(sl)] >= gp.quicksum(x_4[c] for c in sl_key__value_c_mapping_dict[sl]))
    
    
# Previous OBJECTIVE 1: Minimize Waste : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c]*last_two_rows[c]["Wasted Length"] for c in patterns_df) <= ceil(CSProb.objVal *1000)/1000)

# Satisfying Requirements
for l in all_length_of_cuts_allowed:
    CSProb4.addConstr(gp.quicksum(x_4[c] * new_df[c][l] for c in patterns_df) >= requirements[l])

# Previous Objective 2: Minimize Cost : Now treated as constraint
CSProb4.addConstr(gp.quicksum(x_4[c] * DashBoard_Dataset_StandardLengths["Associated Cost"][float(last_two_rows[c]["Standard Max Length"])] for c in patterns_df) <= ceil(CSProb2.objVal *1000)/1000)




# Previous Objective 3: to Minimize the number of Unique Standard Length Usage
CSProb4.addConstr(gp.quicksum(b_4[sl] for sl in unique_elements_row) <= ceil(CSProb3.objVal *1000)/1000)


# New Objective 4 to Minimize the number of Unique Pattern Usage
CSProb4.setObjective(gp.quicksum(y_4[c] for c in patterns_df),sense=GRB.MINIMIZE)





# Set a time limit for optimization (e.g., 60 seconds)
#https://www.gurobi.com/documentation/10.0/refman/python_parameter_examples.html#PythonParameterExamples
currentTIMElimit = CSProb4.Params.timeLimit
print("Default Time Limit: ",currentTIMElimit)
#CSProb4.Params.timeLimit = Timelimit_for_each_final_CSP_problem_minimizing_WCSP


currentIntFeasTolerancelimit = CSProb4.Params.IntFeasTol
print("Default Interger Tolerance: ",currentIntFeasTolerancelimit)
CSProb4.Params.IntFeasTol = 1e-9


#FeasibilityTol
#https://www.gurobi.com/documentation/current/refman/feasibilitytol.html#parameter:FeasibilityTol
currentFeasibilityTolerancelimit = CSProb4.Params.FeasibilityTol
print("Default Feasibility Tolerance: ",currentFeasibilityTolerancelimit)
CSProb4.Params.FeasibilityTol = 1e-9


#OptimalityTol
#https://www.gurobi.com/documentation/current/refman/optimalitytol.html
currentOptimalityTolerancelimit = CSProb4.Params.OptimalityTol
print("Default Optimality Tolerance: ",currentOptimalityTolerancelimit)
CSProb4.Params.OptimalityTol = 1e-9


currentMIPGapLimit = CSProb4.Params.MIPGap
print("Default MIP Gap Limit: ",currentMIPGapLimit)
#CSProb.setParam('MIPGap', 0)
CSProb4.Params.MIPGap = 0
print("Default MIP Gap Limit: ",currentMIPGapLimit)

#https://www.gurobi.com/documentation/current/refman/mip_models.html#:~:text=The%20MIPFocus%20parameter%20allows%20you%20to%20modify%20your,good%20quality%20feasible%20solutions%2C%20you%20can%20select%20MIPFocus%3D1.
#https://www.gurobi.com/documentation/current/refman/mipfocus.html
#https://support.gurobi.com/hc/en-us/community/posts/8791754564369-How-does-MIPFocus-really-work-
currentMIPFocus = CSProb4.Params.MIPFocus
print("The current MIPFocus is: ",currentMIPFocus)
CSProb4.Params.MIPFocus=2


# Optimize the model
CSProb4.optimize()

Default Time Limit:  inf
Default Interger Tolerance:  1e-05
Set parameter IntFeasTol to value 1e-09
Default Feasibility Tolerance:  1e-06
Set parameter FeasibilityTol to value 1e-09
Default Optimality Tolerance:  1e-06
Set parameter OptimalityTol to value 1e-09
Default MIP Gap Limit:  0.0001
Set parameter MIPGap to value 0
Default MIP Gap Limit:  0.0001
The current MIPFocus is:  0
Set parameter MIPFocus to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i3-10110U CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 875 rows, 842 columns and 4274 nonzeros
Model fingerprint: 0x1033189a
Variable types: 0 continuous, 842 integer (424 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+03]
Presolve added 6 rows and 6 columns
Presolve time: 0.0

# Populate Result Excel Sheet

In [33]:
end_time_is_this=time.time()
print(end_time_is_this)

actual_time_taken = end_time_is_this - start_time_is_this
actual_time_taken

1702799381.511226


313.059045791626

In [34]:
winsound.Beep(500,1000)
wb_final=openpyxl.Workbook()

sheet_for_Usage = wb_final.active
sheet_for_Usage.title = 'Variable x and y'
sheet_for_b = wb_final.create_sheet(title='Variable b')
sheet_for_Objectives = wb_final.create_sheet(title='Objectives')

#sheet_for_x
row_number=1
sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage in Objective 1 (x)"
sheet_for_Usage.cell(row=row_number,column=4).value= "No. of Usage in Objective 2 (x_2)"
sheet_for_Usage.cell(row=row_number,column=5).value= "No. of Usage in Objective 3 (x_3)"
sheet_for_Usage.cell(row=row_number,column=6).value= "No. of Usage in Objective 4 (x_4)"
sheet_for_Usage.cell(row=row_number,column=7).value= "Unique Pattern Usages in Objective 4 (y_4)"

for c in patterns_df:
    row_number+=1
    sheet_for_Usage.cell(row=row_number,column=1).value=c
    sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
    sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
    sheet_for_Usage.cell(row=row_number,column=4).value=x_2[c].x
    sheet_for_Usage.cell(row=row_number,column=5).value=x_3[c].x
    sheet_for_Usage.cell(row=row_number,column=6).value=x_4[c].x
    sheet_for_Usage.cell(row=row_number,column=7).value=y_4[c].x
    #if x_3[c].x:
    #    print(c, " used " , x_3[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])


    
#sheet_for_b
row_number=1
sheet_for_b.cell(row=row_number,column=1).value= "Standard Length"
sheet_for_b.cell(row=row_number,column=2).value= "Utilization in Objective 3 (b-3)"
sheet_for_b.cell(row=row_number,column=3).value= "Utilization in Objective 4 (b_4)"

for sl in unique_elements_row:
    row_number+=1
    sheet_for_b.cell(row=row_number,column=1).value=sl
    sheet_for_b.cell(row=row_number,column=2).value=b_3[sl].x
    sheet_for_b.cell(row=row_number,column=3).value=b_4[sl].x
    
    
#sheet_for_Objectives
row_number=1
sheet_for_Objectives.cell(row=row_number,column=1).value= "Objective 1"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Objective 2"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Objective 3"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Objective 4"
sheet_for_Objectives.cell(row=row_number,column=5).value= "Total Time taken for the Pattern Generations and all Objective Computations"
row_number=2
sheet_for_Objectives.cell(row=row_number,column=1).value= "Minimum Total WASTE"
sheet_for_Objectives.cell(row=row_number,column=2).value= "Minimum Total COST"
sheet_for_Objectives.cell(row=row_number,column=3).value= "Minimum Unique STANDARD LENGTH Usage"
sheet_for_Objectives.cell(row=row_number,column=4).value= "Minimum Unique PATTERN Usage"
sheet_for_Objectives.cell(row=row_number,column=5).value= "(Seconds)"
row_number=3
sheet_for_Objectives.cell(row=row_number,column=1).value= CSProb.objVal
sheet_for_Objectives.cell(row=row_number,column=2).value= CSProb2.objVal
sheet_for_Objectives.cell(row=row_number,column=3).value= CSProb3.objVal
sheet_for_Objectives.cell(row=row_number,column=4).value= CSProb4.objVal
sheet_for_Objectives.cell(row=row_number,column=5).value= actual_time_taken


Final_output_file_path = "./Prob"+str(Decisions["Problem"])+"_PatternUsage_with_WCSP_Minimization"+".xlsx"
wb_final.save(Final_output_file_path)

In [35]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x[c].x
#     if x[c].x:
#         print(c, " used " , x[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])

        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Trim Loss"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb.objVal
# print("Objective Value:", CSProb.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumTrimLoss"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)

In [36]:
# winsound.Beep(500,1000)
# wb_final=openpyxl.Workbook()

# sheet_for_Usage = wb_final.active

# row_number=1
# sheet_for_Usage.cell(row=row_number,column=1).value= "PATTERN"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Standard Length"
# sheet_for_Usage.cell(row=row_number,column=3).value= "No. of Usage"


# for c in patterns_df:
#     row_number+=1
#     sheet_for_Usage.cell(row=row_number,column=1).value=c
#     sheet_for_Usage.cell(row=row_number,column=2).value=last_two_rows[c]["Standard Max Length"]
#     sheet_for_Usage.cell(row=row_number,column=3).value=x_2[c].x
#     if x_2[c].x:
#         print(c, " used " , x_2[c].x , "times, with standard length", last_two_rows[c]["Standard Max Length"])
        
# row_number+=3
# sheet_for_Usage.cell(row=row_number,column=1).value= "Minimum Total Cost"
# sheet_for_Usage.cell(row=row_number,column=2).value= "Objective Value:"
# sheet_for_Usage.cell(row=row_number,column=3).value= CSProb2.objVal
# print("Objective Value:", CSProb2.objVal)

# Final_output_file_path = "./Problem_"+str(Decisions["Problem"])+"__PatternUsage_withMinimumCost"+"_Gurobi.xlsx"
# wb_final.save(Final_output_file_path)